In [143]:
import numpy as np
import pandas as pd
import os
from datetime import date
import pymysql
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

#### Data Base Connectivity

In [154]:
# data base connection using pymysql module
def dBCon_Maria():
    connection = pymysql.connect(host='localhost',
                            user='root',
                            password='password',
                            db='data_dashboard')
    return connection

connection = dBCon_Maria()
cursor=connection.cursor()

#### Select Instance for extracting feeding data

In [155]:
tbl_lookup = pd.read_sql("SELECT * FROM tbl_pbr_lookup", connection)
tbl_lookup

,instance,unit,species,start_date,end_date,remark,raw_file_name
0,1,6,1,1629468000,1633939195,missing values from 2021-10-3 2:56:55 to 2021-10-3 9:27:55. Reason: yet to update,_1_FPC13_ChCal_1.csv
1,2,6,1,1634652000,1644488995,None,_2_FPC13_ChCal_2.csv
2,3,6,11,1625227200,1628157595,None,_3_FPC13_ThalaPs.csv
3,4,1,2,1614763800,1618392595,None,_4_FPC14_chamu.csv
4,5,1,7,1625486400,1639564200,None,_5_FPC14_RhoSa.csv
5,6,1,11,1618561800,1625228995,None,_6_FPC14_thalaps.csv
6,7,4,2,1634914800,1646645395,None,_7_FPC21_chamu.csv
7,8,4,9,1623051000,1626771600,None,_8_FPC21_Techu.csv
8,9,3,3,1620216000,1634632200,None,_9_FPC22_DiaLut.csv
9,10,5,7,1616427000,1626690600,None,_10_FPC23_Rhosa.csv


In [156]:
###### ================================= SELECT THE INSTANCE TO EXTRACT AND UPLOAD FEEDING DATA ============================
set_instance = 2

#select the log extract parameters
instance_data = tbl_lookup.query('instance == @set_instance')

In [157]:
unit_no = int(instance_data['unit'])
unit_name = str(pd.read_sql("SELECT unit_name FROM tbl_unit WHERE unit_id = %s", connection, params=[unit_no])['unit_name'][0])
start_time = int(instance_data['start_date'])
end_time = int(instance_data['end_date'])
file_name = str(instance_data['raw_file_name'][set_instance-1])
# Log variable definitions
unit_ffu = 2

#### Load feed data for selected instance from database

In [158]:
feed_instance = pd.read_sql("SELECT * FROM tbl_log_data WHERE time_epoch >= %s AND time_epoch <= %s", connection, params=[start_time, end_time])
feed_instance['Time_Micro'] = feed_instance['time_epoch'] + (feed_instance['MicroSec']/1000000)
feed_tmp1 = feed_instance.query('unit == @unit_ffu and category == 2 and parameter == @prm_t10')
feed_tmp2 = feed_tmp1[feed_tmp1.apply(lambda r: r.str.contains(unit_name).any(), axis=1)]
feed_tmp3 = feed_tmp2[~feed_tmp2.message.str.contains("ended")]
feed_tmp3['Time_micro'] = feed_tmp3['time_epoch'] + (feed_tmp3['MicroSec']/1000000)
time_fedd = feed_tmp3['Time_micro'].to_list()
it = iter(time_fedd)
time_tuples = [*zip(it, it)]  # list(zip(it, it))

appended_feed_data = []
for item in time_tuples:
    data = feed_instance.query('(Time_Micro >= @item[0] and Time_Micro < @item[1]) and unit ==@unit_ffu and category == 2')
    # store DataFrame in list
    appended_feed_data.append(data)
# see pd.concat documentation for more info
appended_feed_data = pd.concat(appended_feed_data)

recipe_tmp2 = appended_feed_data[appended_feed_data['parameter'].str.startswith('T')]
recipe_tmp3 = recipe_tmp2[recipe_tmp2['message'].str.contains('ended')]

recipe_tmp3['message'] = recipe_tmp3['message'].str.replace(r'<.*?>.*?</.*?>', '', regex=True)
recipe_tmp3['message'] = recipe_tmp3['message'].str.replace('Dosing ended for ', '', regex=True)
recipe_tmp3['message'] = recipe_tmp3['message'].str.replace('to Tank = FFU_T10', '', regex=True)
recipe_tmp3['message'] = recipe_tmp3['message'].str.replace(r'\([^)]*\)', '', regex=True)

exclude = ['low level', 'NaOH', 'FFU_waste', 'Adjusting','sequence', 'Reactor','Stock to Tank']
recipe_tmp3 = recipe_tmp3[~recipe_tmp3.message.str.contains('|'.join(exclude))]

connection.close()
recipe_tmp3['message'].unique()

array(['NaNO3_2.47M  ', 'KCl_1M  ', 'CaCl2 0.67M  ', 'MgSO4_2M  ',
       'H3B03_0,0625M  ', 'NaCl_4,3M  ', 'Na2SeO3 0.001 mM  ',
       'FeSO4/EDTA 12.05 mM  ', 'KH2PO4_0.157M  ', 'Vitamins 2000xF  ',
       'Micronutrients 400xF  ', 'CompWater  ',
       'Na2SiO3_0,212M/EDTA_0,02_fromStock  ', 'HCl_2M  '], dtype=object)

In [170]:
recipe_tmp4= recipe_tmp3.drop(['MicroSec', 'category', 'Value1', 'unit', 'parameter', 'Time_Micro'],1)
recipe_tmp4['Date'] = pd.to_datetime(recipe_tmp4['date_time']).dt.date
recipe_tmp4 = recipe_tmp4.astype({"Value2": float})
recipe_tmp6 = recipe_tmp4.pivot_table(index=['Date'], columns='message', values='Value2', aggfunc=np.sum).reset_index().rename_axis(None, axis=1)
recipe_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Date                                  86 non-null     object 
 1   CaCl2 0.67M                           86 non-null     float64
 2   CompWater                             86 non-null     float64
 3   FeSO4/EDTA 12.05 mM                   86 non-null     float64
 4   H3B03_0,0625M                         86 non-null     float64
 5   HCl_2M                                86 non-null     float64
 6   KCl_1M                                86 non-null     float64
 7   KH2PO4_0.157M                         86 non-null     float64
 8   MgSO4_2M                              86 non-null     float64
 9   Micronutrients 400xF                  86 non-null     float64
 10  Na2SeO3 0.001 mM                      86 non-null     float64
 11  Na2SiO3_0,212M/EDTA_0

In [171]:
recipe_table = recipe_tmp6.fillna(0)
recipe_table = recipe_table.rename(columns=lambda x: re.sub(' ','',x))


### Export the Recipe data for each instance to separate database

In [167]:
import sqlalchemy
def dbConn_sqlAlch():
    try:
        database_username = 'root'
        database_password = 'password'
        database_ip       = '127.0.0.1:3306'
        database_name     = 'data_recipe'
        database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                                    format(database_username, database_password, 
                                                            database_ip, database_name))
        return database_connection
    except Exception as e: print(e)


database_connection = dbConn_sqlAlch()

In [172]:
recipe_name='tbl_{}_{}_recipe'.format(set_instance, unit_name)

In [173]:
# load content of the export_list into list split by new line
# data = [line.strip() for line in open("exported_list.txt", 'r')]
with open('./exported_list.txt') as f:
    lines = f.read().splitlines()

if (recipe_name not in lines):
    try:
        recipe_table.to_sql(con=database_connection, name='tbl_{}_{}_recipe'.format(set_instance, unit_name), if_exists='append', index=False)
        file1 = open("exported_list.txt", "a")
        file1.write("\n")     
        file1.write(recipe_name)
        file1.close()
        print('file is exported')
    except Exception as e: print(e)

else:
    print('data is present in the db - check and very the trial number in tbl_pbr_data')

file is exported


In [135]:
try:
    recipe_table.to_sql(con=database_connection, name='tbl_{}_{}_recipe'.format(set_instance, unit_name), if_exists='append', index=False)
    
except Exception as e: print(e)